# Modern Monte Carlo Tree Search

How can we efficiently search without relying on expert knowledge?

- **exploration**: learn values of actions we are uncertain about
- **exploitation**: focus search on promising parts of tree



## Multi-Armed bandit

- MDP with no state
- we want to minimize the total regret (how much reward you lost by not selecting optimal action up to time T)

- **information state search**: exploration provides information which can increase expected reward in future 
- optimal solutions can be found by solving infinte-state MDP over info states
- need heuristics to make things tractable

bandit strategies scale with how we handle uncertainty:

- no exploration, just greedily pull best values we have seen in past
- random exploration
- explore with preference towards uncertaint



### $\epsilon$-Greedy Algorithm

action value is just average of rewards we have seen over time

e-Greedy just selects the non-max at random sometimes


### UCB Algorithm

e-Greedy not awesome because a lot of the time we end up exploring bad actions that we sort of know will not yield anything great

The idea is to agument the value function with an upper bound on the reward value so that the true reward value is lower: $Q(a) \leq \hat{Q}_t(a) + \hat{U}_t(a)$.

Hoeffding's inequality can be used if we do not want to assign any prior to the distribution of rewards, works for any bounded distribution. 

$$P[\frac{1}{n} \sum_{i =1}^{n}(Z_i - E[Z_i]) \geq \delta\ ] \leq e^{- \frac{2n\delta^2}{(b - a)^2}}$$

So we view this average of the random variable $Z_i$ as the averaged reward of a particular action, $\hat{Q_t}(a) = \frac{1}{n} \sum_{t=0}^n R_i$. The inequality then gets written as (with $t = N_t(a)$, the number of times this action has been visited): 

$$P[Q_t - \hat{Q_t} \geq \delta ] \leq e^{-\frac{2N_t(a)\delta^2}{(b-a)^2}}$$

We see that then our upper bound on the true reward can be given by $\delta = \hat{U_t}(a)$. And if we make the simplifying assumption that the reward is bounded to be on the interval $(0, 1)$. We can then for any value of the probability of deviation, $p$, we can solve for what we expect the upper bound on that deviation to be:

$$
p \approx e^{-2tU_t(a)^2} \\
U_t(a) = \sqrt{\frac{- \log p}{2 N_t (a)}}
$$

### UCB 1
Most of the time when we see the UCB algorithm in practice (like monte carlo tree search), we see it in the guise of 
### 